In [118]:
from __future__ import annotations

import json
import os
from pathlib import Path
from queue import Queue

import numpy as np
import pandas as pd
from librosa import load, feature
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

PATH_TO_FILES = Path('files/')

In [119]:
class FileFinder:
    suffixes: set[str]
    _queue: Queue[Path]

    def __init__(self, root, suffixes):
        self.root = Path(root)
        self._queue = Queue()
        self.suffixes = suffixes

        self._queue.put(self.root)

    def get_next_file(self):
        if self._queue.empty():
            print("Queue is empty")
            return None

        res = self._queue.get()
        if res.is_dir():
            for child in res.iterdir():
                if child.suffix[1:].lower() in self.suffixes or child.is_dir():
                    # print(f'Added {child.name} to queue [{"folder" if child.is_dir() else child.suffix}]')
                    self._queue.put(child)
            return Ellipsis
        else:
            return res if res.suffix[1:] in self.suffixes else None

    def reset(self):
        self._queue = Queue()
        self._queue.put(self.root)

    def get_rel_path(self, file: Path):
        return Path(os.path.relpath(file.parent, self.root).__str__())

    def get_full_path(self, file: Path):
        return self.root.joinpath(file)

class FileIO:
    _data: list[dict, ...]

    def __init__(self, path):
        self.path = Path(path)
        self._data = ...
    
    def get_data(self):
        with open(self.path) as json_file:
            self._data = json.load(json_file)
        return self._data
    
    def set_data(self, data):
        if type(data) is list:
            self._data = data
            with open(self.path, 'w') as json_file:
                json_file.write(json.dumps(self._data, sort_keys=False, indent=4))
        

In [133]:
#TODO: standardize/normalize features

class FeatureExtractor:
    def __init__(self):
        self.mfccs = 10
        self.rate = 22050
    
    def not_static(self):
        pass
    
    def mfcc(self, audio):
        return np.mean(feature.mfcc(y=audio, sr=self.rate, n_mfcc=self.mfccs).tolist(), axis=1)
    
    def contrast(self, audio):
        return np.mean(feature.spectral_contrast(y=audio, sr=self.rate).tolist())
    
    def zcr(self, audio):
        self.not_static()
        return np.mean(feature.zero_crossing_rate(y=audio).tolist())
        

In [134]:
finder = FileFinder(PATH_TO_FILES, ('au', ))
output = FileIO('data.json')
extractor = FeatureExtractor()
data = []

while True:
    curr = finder.get_next_file()
    if not curr:
        break
    elif curr is ...:
        continue
    else:
        curr_file, fs = load(curr, duration=7)
        data.append(dict(
            id=curr.name,
            genre=curr.parent.name,
            **dict(zip((f'mfcc{n}' for n in range(extractor.mfccs)), extractor.mfcc(curr_file))),
            contrast=extractor.contrast(curr_file),
            zcr=extractor.zcr(curr_file),
            prediction=''
        ))
        # print(curr.parent.name, curr.name.split('.')[1])

output.set_data(data)
print('Success')

Queue is empty
Success


In [137]:
features = [*(f'mfcc{n}' for n in range(extractor.mfccs // 2)), 'zcr']

frame = pd.DataFrame(data)
categories = frame[['genre']]
predictors = frame[features]
predictors.columns = features

pre_train, pre_test, cat_train, cat_test = train_test_split(predictors, categories, test_size=0.33, random_state=5)
model = LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=1000)
model.fit(pre_train, cat_train.genre)
predictions = model.predict(pre_test)
# confusion_matrix(cat_test, predictions)
print(classification_report(cat_test, predictions))

              precision    recall  f1-score   support

   classical       1.00      0.73      0.84        11
      hiphop       0.63      0.60      0.62        20
        jazz       0.58      0.88      0.70        16
         pop       0.57      0.81      0.67        16
      reggae       0.78      0.35      0.48        20

    accuracy                           0.65        83
   macro avg       0.71      0.67      0.66        83
weighted avg       0.69      0.65      0.64        83

